### Load data & packages

### Packages

In [1]:
import numpy as np
import pandas as pd
import seaborn

### Data

In [2]:
#encoding: utf-8
#encoding: iso-8859-1
#encoding: win-1252


# raw data
#path2019 = "~/Documents/UFABC/Disciplinas/Mineração de Dados/Dados para o Projeto/SINASC_2015-2019/ETLSINASC.DNRES_SP_2019_t.csv"
#path2018 = "~/Documents/UFABC/Disciplinas/Mineração de Dados/Dados para o Projeto/SINASC_2015-2019/ETLSINASC.DNRES_SP_2018_t.csv"
#path2017 = "~/Documents/UFABC/Disciplinas/Mineração de Dados/Dados para o Projeto/SINASC_2015-2019/ETLSINASC.DNRES_SP_2017_t.csv"
#path2016 = "~/Documents/UFABC/Disciplinas/Mineração de Dados/Dados para o Projeto/SINASC_2015-2019/ETLSINASC.DNRES_SP_2016_t.csv"
#path2015 = "~/Documents/UFABC/Disciplinas/Mineração de Dados/Dados para o Projeto/SINASC_2015-2019/ETLSINASC.DNRES_SP_2019_t.csv"

# filtered data
path2019 = "~/Documents/jupyter-workplace/data-mining-assignment/filtered_data2019.csv"
path2018 = "~/Documents/jupyter-workplace/data-mining-assignment/filtered_data2018.csv"
path2017 = "~/Documents/jupyter-workplace/data-mining-assignment/filtered_data2017.csv"
path2016 = "~/Documents/jupyter-workplace/data-mining-assignment/filtered_data2016.csv"
path2015 = "~/Documents/jupyter-workplace/data-mining-assignment/filtered_data2015.csv"
pathcbo =  "~/Documents/jupyter-workplace/data-mining-assignment/CBO2002.csv"

data2019 = pd.read_csv(path2019, index_col=0)
data2018 = pd.read_csv(path2018, index_col=0)
data2017 = pd.read_csv(path2017, index_col=0)
data2016 = pd.read_csv(path2016, index_col=0)
data2015 = pd.read_csv(path2015, index_col=0)

cbo = pd.read_csv(pathcbo, index_col=0, encoding = 'ISO-8859-1')

## Filter

### Filter variables

In [3]:
# Idade Mae; Estado Civil; Escolaridade; Etnia/Cor; Municipio de Residencia; Ocupação
perfil_materno = ["IDADEMAE","ESTCIVMAE","ESCMAE","RACACOR","res_MUNNOME","CODOCUPMAE"]

# Idade Gestacional; Consultas de Pre natal; Mês que iniciou o prenatal
perfil_gest = ["GESTACAO","CONSULTAS","MESPRENAT"]

# APGAR; Peso ao nascimento; Anomalia congenita; prematuridade;
perfil_nasc = ["APGAR1","APGAR5","PESO","IDANOMAL","parto_prematuro"]

filt = perfil_materno+perfil_gest+perfil_nasc

In [4]:
# filtering
data2019 = data2019[filt]
data2018 = data2018[filt]
data2017 = data2017[filt]
data2016 = data2016[filt]
data2015 = data2015[filt]

In [ ]:
# saving files
#data2019.to_csv('filtered_data2019.csv')
#data2018.to_csv('filtered_data2018.csv') 
#data2017.to_csv('filtered_data2017.csv') 
#data2016.to_csv('filtered_data2016.csv') 
#data2015.to_csv('filtered_data2015.csv') 

### Filter for sampling

Filter by city!

In [5]:
import warnings
warnings.filterwarnings('ignore')

# filter são paulo!
data2019_sp = data2019[data2019['res_MUNNOME'] == "São Paulo"]
data2018_sp = data2018[data2018['res_MUNNOME'] == "São Paulo"]
data2017_sp = data2017[data2017['res_MUNNOME'] == "São Paulo"]
data2016_sp = data2016[data2016['res_MUNNOME'] == "São Paulo"]
data2015_sp = data2015[data2015['res_MUNNOME'] == "São Paulo"]

Checking the filtered data...

In [ ]:
print("2019\n\n")

print("Municipio: \n",data2019_sp['res_MUNNOME'].describe(),"\n")

## Process data

### Binding years

In [6]:
#concatenate
frames = [data2015_sp,data2016_sp,data2017_sp,data2018_sp,data2019_sp]
all_data = pd.concat(frames)

all_data = all_data.drop(['res_MUNNOME'], 1)

all_data.shape

(819131, 13)

 ### Reformating data types

In [7]:
data_updated = all_data
data_updated.columns

Index(['IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'RACACOR', 'CODOCUPMAE', 'GESTACAO',
       'CONSULTAS', 'MESPRENAT', 'APGAR1', 'APGAR5', 'PESO', 'IDANOMAL',
       'parto_prematuro'],
      dtype='object')

In [8]:
# CBO data frame
cbo2 = {'codigo':  list(cbo.index), 'titulo': list(cbo["TITULO"])}
cbo2 = pd.DataFrame(data = cbo2)
dic_cbo = cbo2.set_index('codigo').T.to_dict('list')

data_updated['CODOCUPMAE'] = data_updated['CODOCUPMAE'].map(dic_cbo)

data_updated.head(n = 5)

,IDADEMAE,ESTCIVMAE,ESCMAE,RACACOR,CODOCUPMAE,GESTACAO,CONSULTAS,MESPRENAT,APGAR1,APGAR5,PESO,IDANOMAL,parto_prematuro
1,26,2,2,4,NaN,5,3,3.0,9.0,10.0,3165.0,2,0.0
1,22,1,4,4,[Técnico de planejamento de produção],5,4,5.0,8.0,10.0,3240.0,2,0.0
1,24,1,4,4,NaN,5,4,2.0,9.0,10.0,3250.0,2,0.0
1,27,2,5,4,[Advogado],5,4,NaN,10.0,10.0,2345.0,2,2.0
1,17,1,4,4,NaN,5,3,4.0,9.0,10.0,2874.0,2,0.0


In [9]:
# flatten CODOCUPMAE
replaced = data_updated['CODOCUPMAE'].astype(object).replace(np.nan, 0)
replaced = list(replaced)
for i in range(len(replaced)): 
    if replaced[i] == 0:
        replaced[i] = [replaced[i]]

flatten_ocupation = list(np.concatenate(replaced).flat)

data_updated['CODOCUPMAE'] = flatten_ocupation

data_updated.head(n = 5)

,IDADEMAE,ESTCIVMAE,ESCMAE,RACACOR,CODOCUPMAE,GESTACAO,CONSULTAS,MESPRENAT,APGAR1,APGAR5,PESO,IDANOMAL,parto_prematuro
1,26,2,2,4,0,5,3,3.0,9.0,10.0,3165.0,2,0.0
1,22,1,4,4,Técnico de planejamento de produção,5,4,5.0,8.0,10.0,3240.0,2,0.0
1,24,1,4,4,0,5,4,2.0,9.0,10.0,3250.0,2,0.0
1,27,2,5,4,Advogado,5,4,NaN,10.0,10.0,2345.0,2,2.0
1,17,1,4,4,0,5,3,4.0,9.0,10.0,2874.0,2,0.0


In [10]:
# creating APGAR unified variable (MEAN)

apgar = (data_updated['APGAR1'] + data_updated['APGAR5'])/2

data_updated['APGAR'] = apgar

data_updated = data_updated.drop(['APGAR1','APGAR5'], 1)

# remover values 99
data_updated = data_updated[data_updated['APGAR'] < 11]

data_updated

,IDADEMAE,ESTCIVMAE,ESCMAE,RACACOR,CODOCUPMAE,GESTACAO,CONSULTAS,MESPRENAT,PESO,IDANOMAL,parto_prematuro,APGAR
1,26,2,2,4,0,5,3,3.0,3165.0,2,0.0,9.5
1,22,1,4,4,Técnico de planejamento de produção,5,4,5.0,3240.0,2,0.0,9.0
1,24,1,4,4,0,5,4,2.0,3250.0,2,0.0,9.5
1,27,2,5,4,Advogado,5,4,NaN,2345.0,2,2.0,10.0
1,17,1,4,4,0,5,3,4.0,2874.0,2,0.0,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...
1,35,1,5,1,Administrador,5,3,1.0,3172.0,2,0.0,9.5
1,31,5,5,1,Publicitário,5,4,3.0,3480.0,2,0.0,9.5
1,33,1,5,2,0,5,3,6.0,2940.0,2,0.0,8.5
1,33,2,5,4,0,5,4,2.0,2770.0,2,0.0,9.5


In [11]:
# mapping

data_updated['ESTCIVMAE'] = data_updated['ESTCIVMAE'].map({1:'Solteira',2:'Casada',3:'Viuva', 
                                                           4:'Divorciada',5:'Uniao consesual',9:'NA'})

data_updated['ESCMAE'] = data_updated['ESCMAE'].map({1:'Nenhuma', 2:'1 a 3 anos', 3:'4 a 7 anos',
                                                    4:'8 a 11 anos', 5:'12 e mais',9:'NA'})

data_updated['RACACOR'] = data_updated['RACACOR'].map({1:'Branca',2:'Preta',3:'Amarela',4:'Parda',5:'Indigina'})

data_updated['IDANOMAL'] = data_updated['IDANOMAL'].map({1:'Sim',2:'Nao',9:'NA'})

data_updated['parto_prematuro'] = data_updated['parto_prematuro'].map({0:'Normal',1:'Indicio',2:'Indicio',
                                                                       3:'Prematuro'})

In [12]:
# mapping intervals apgar (7 < x) (8 > x > 3) (4 > x)
normal = [10, 9.5, 9, 8.5, 8]
preocupante = [7.5, 7, 6.5, 6, 5.5, 5, 4.5, 4]
problematico = [3.5, 3, 2.5, 2, 1.5, 1, 0.5, 0]

data_updated['APGAR'] = data_updated['APGAR'].astype("float").replace(problematico, 2)
data_updated['APGAR'] = data_updated['APGAR'].astype("float").replace(preocupante, 1)
data_updated['APGAR'] = data_updated['APGAR'].astype("float").replace(normal, 0)

data_updated['APGAR'] = data_updated['APGAR'].map({0: "Normal", 1: "Preocupante", 2:"Problematico"})

In [13]:
# data types
data_updated['IDADEMAE'].astype('int')

estciv_dtype = pd.CategoricalDtype(categories=['Solteira', 'Casada','Uniao consesual','Divorciada','Viuva'], 
                                   ordered=False)
data_updated['ESTCIVMAE'].astype(estciv_dtype)

esc_dtype = pd.CategoricalDtype(categories=['Nenhuma', '1 a 3 anos','4 a 7 anos','8 a 11 anos','12 e mais'], 
                                ordered=True)
data_updated['ESCMAE'].astype(esc_dtype)

raca_dtype = pd.CategoricalDtype(categories=['Branca', 'Preta','Amarela','Parda','Indigina'], 
                                   ordered=False)
data_updated['RACACOR'].astype(raca_dtype)

anom_dtype = pd.CategoricalDtype(categories=['Sim', 'Nao'], 
                                      ordered=False)
data_updated['IDANOMAL'].astype(anom_dtype)

prematuro_dtype = pd.CategoricalDtype(categories=['Normal', 'Indicio','Prematuro'], 
                                      ordered=False)
data_updated['parto_prematuro'].astype(esc_dtype)

APGAR_dtype = pd.CategoricalDtype(categories=['Normal', 'Preocupante','Problematico'], 
                                  ordered=False)
data_updated['APGAR'].astype(APGAR_dtype)

1    Normal
1    Normal
1    Normal
1    Normal
1    Normal
      ...  
1    Normal
1    Normal
1    Normal
1    Normal
1    Normal
Name: APGAR, Length: 815907, dtype: category
Categories (3, object): ['Normal', 'Preocupante', 'Problematico']

In [14]:
# removendo Ocupação == 0
data_updated = data_updated[data_updated['CODOCUPMAE'] != '0']

In [15]:
data_updated

,IDADEMAE,ESTCIVMAE,ESCMAE,RACACOR,CODOCUPMAE,GESTACAO,CONSULTAS,MESPRENAT,PESO,IDANOMAL,parto_prematuro,APGAR
1,22,Solteira,8 a 11 anos,Parda,Técnico de planejamento de produção,5,4,5.0,3240.0,Nao,Normal,Normal
1,27,Casada,12 e mais,Parda,Advogado,5,4,NaN,2345.0,Nao,Indicio,Normal
1,31,Casada,8 a 11 anos,Parda,Vendedor de comércio varejista,5,4,3.0,3290.0,Nao,Normal,Normal
1,21,Uniao consesual,12 e mais,Amarela,Operador de caixa,5,4,4.0,2572.0,Nao,Normal,Normal
1,25,Solteira,4 a 7 anos,Parda,Trabalhador volante da agricultura,5,4,2.0,3360.0,Nao,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...
1,31,Casada,12 e mais,Branca,Supervisor administrativo,4,4,1.0,2490.0,Nao,Prematuro,Preocupante
1,25,Uniao consesual,8 a 11 anos,Branca,Atendente de farmácia - balconista,5,3,3.0,3300.0,Nao,Normal,Normal
1,34,Casada,12 e mais,Branca,Fisioterapeuta geral,5,4,1.0,3500.0,Nao,Normal,Normal
1,35,Solteira,12 e mais,Branca,Administrador,5,3,1.0,3172.0,Nao,Normal,Normal


##### Saving dataframe

In [16]:
data_updated.to_csv('reformed_all_data.csv')

In [17]:
data_updated.shape

(344598, 12)